<a href="https://colab.research.google.com/github/sam134-eng/INM706-Sequence-analysis/blob/main/Sentiment_Analysis_Test2_GloVe_FIXED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
import nltk
nltk.download('punkt')

import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("Tweets.csv")
df = df[['text', 'airline_sentiment']]
df = df[df['airline_sentiment'].isin(['positive', 'neutral', 'negative'])]
df['label'] = df['airline_sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2})

def clean_text(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    return text.lower().strip()

df["clean_text"] = df["text"].apply(clean_text)


Saving Tweets.csv to Tweets.csv


In [3]:
!pip install nltk
import nltk
# Download the necessary NLTK resources, including punkt_tab
nltk.download('punkt')
nltk.download('punkt_tab')

import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch


# First, tokenize the cleaned text
df['tokens'] = df['clean_text'].apply(word_tokenize)

# Create a vocabulary and word-to-index mapping
all_tokens = [token for tokens in df['tokens'] for token in tokens]
vocab = Counter(all_tokens)
# Add a padding token at index 0
word_to_index = {'<pad>': 0}
word_to_index.update({word: i + 1 for i, (word, _) in enumerate(vocab.most_common())})

# Convert tokens to indices
df['indexed_tokens'] = df['tokens'].apply(lambda tokens: [word_to_index.get(token, word_to_index.get('<unk>', 0)) for token in tokens])

# Pad the sequences
# Determine the maximum sequence length
max_seq_length = max(len(x) for x in df['indexed_tokens'])

# Pad or truncate sequences to the maximum length
def pad_sequences(sequences, maxlen, padding='post', value=0):
    padded_sequences = []
    for seq in sequences:
        if len(seq) < maxlen:
            if padding == 'post':
                padded_seq = seq + [value] * (maxlen - len(seq))
            else: # padding == 'pre'
                 padded_seq = [value] * (maxlen - len(seq)) + seq
        else:
            padded_seq = seq[:maxlen]
        padded_sequences.append(padded_seq)
    return torch.tensor(padded_sequences, dtype=torch.long)

X = pad_sequences(df['indexed_tokens'], max_seq_length)
y = torch.tensor(df['label'].values, dtype=torch.long)


# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Create DataLoader instances
train_loader = DataLoader(SentimentDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(SentimentDataset(X_test, y_test), batch_size=32)

# Sanity check: Ensure indices are within embedding range
# Use X instead of encoded_texts, as X contains the padded indexed tokens
max_id = X.max().item()
print(f"Max token index: {max_id}, Vocab size: {len(vocab)}")

Max token index: 12559, Vocab size: 12559


In [ ]:
# implement GloVe Embedding Integration
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

embedding_dim = 100
glove_path = "glove.6B.100d.txt"

glove_embeddings = {}
with open(glove_path, 'r', encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split()
        word = parts[0]
        vector = np.array(parts[1:], dtype='float32')
        glove_embeddings[word] = vector

# Change here: Use len(word_to_index) for the embedding matrix size
embedding_matrix = np.zeros((len(word_to_index), embedding_dim))
# Iterate through word_to_index to correctly map words to indices
for word, idx in word_to_index.items():
    # Skip the padding token as its vector is already initialized to zeros
    if word != '<pad>':
        vector = glove_embeddings.get(word)
        if vector is not None:
            embedding_matrix[idx] = vector

--2025-05-24 20:46:56--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-05-24 20:46:56--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-05-24 20:46:56--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [ ]:
class_counts = df['label'].value_counts().sort_index().values
weights = 1.0 / torch.tensor(class_counts, dtype=torch.float)
from sklearn.utils.class_weight import compute_class_weight

# Compute weights from y_train labels
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(y_train),
                                     y=y_train)

weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim, embedding_matrix):
        super(LSTMClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(embedding_matrix), freeze=False, padding_idx=0)
        self.lstm = nn.LSTM(
    embed_dim,
    hidden_dim,
    num_layers=2,              #  Increase number of layers here
    batch_first=True,
    dropout=0.3                # added Dropout added
)

        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        _, (hidden, _) = self.lstm(x)
        return self.fc(hidden[-1])

model = LSTMClassifier(len(vocab), 100, 128, 3, embedding_matrix)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
train_losses = []
for epoch in range(10):
    model.train()
    total_loss = 0
    for texts, labels in train_loader:
        outputs = model(texts)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    train_losses.append(avg_loss)
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")

plt.plot(train_losses)
plt.title("Training Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


In [ ]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for texts, labels in test_loader:
        outputs = model(texts)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=["Negative", "Neutral", "Positive"]))
cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Negative", "Neutral", "Positive"], yticklabels=["Negative", "Neutral", "Positive"])
plt.title("Confusion Matrix - LSTM with GloVe")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()
